
# Swing Strategy Scanner — Williams %R + CCI + EMA (NIFTY500-ready)

This notebook scans a list of stocks (e.g., the NIFTY500 universe) and filters those that currently satisfy the swing-trading strategy based on:
- **Trend**: 50-day EMA
- **Momentum Extremes**: Williams %R (14)
- **Deviation Confirmation**: CCI (20)

It produces two outputs:
- `long_candidates.csv` — stocks meeting **long** setup conditions
- `short_candidates.csv` — stocks meeting **short** setup conditions

> The rules are implemented exactly as described in your uploaded strategy note.  fileciteturn0file0


## 1) Setup — install & imports

In [1]:

# If needed, uncomment and run:
# %pip install pandas numpy yfinance

import math, sys, os, warnings, json, time, datetime as dt
from dataclasses import dataclass
from typing import List, Tuple, Optional, Dict

import numpy as np
import pandas as pd
import yfinance as yf  # data


## 2) Parameters — universe & indicator settings

In [2]:

# === Universe ===
# Put your stock list here; for NSE, use '.NS' (e.g., RELIANCE.NS)
TICKERS = ['ABB.NS', 'ADANIENSOL.NS', 'ADANIGREEN.NS', 'ADANIPOWER.NS', 'AMBUJACEM.NS', 'DMART.NS', 'BAJAJHLDNG.NS', 'BAJAJHFL.NS', 'BANKBARODA.NS', 'BPCL.NS', 'BOSCHLTD.NS', 'BRITANNIA.NS', 'CGPOWER.NS', 'CANBK.NS', 'CHOLAFIN.NS', 'DLF.NS', 'DABUR.NS', 'DIVISLAB.NS', 'GAIL.NS', 'GODREJCP.NS', 'HAVELLS.NS', 'HAL.NS', 'HYUNDAI.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'INDHOTEL.NS', 'IOC.NS', 'IRFC.NS', 'NAUKRI.NS', 'INDIGO.NS', 'JSWENERGY.NS', 'JINDALSTEL.NS', 'LTIM.NS', 'LICI.NS', 'LODHA.NS', 'PIDILITIND.NS', 'PFC.NS', 'PNB.NS', 'RECLTD.NS', 'MOTHERSON.NS', 'SHREECEM.NS', 'SIEMENS.NS', 'SWIGGY.NS', 'TVSMOTOR.NS', 'TATAPOWER.NS', 'TORNTPHARM.NS', 'UNITDSPR.NS', 'VBL.NS', 'VEDL.NS', 'ZYDUSLIFE.NS']

HISTORY_START = "2024-01-01"
MIN_BARS = 250  # require at least this many bars after indicators

# === Indicators ===
EMA_LEN = 50      # trend filter
WR_LEN  = 14      # Williams %R
CCI_LEN = 20      # CCI
RECENT_LOOKBACK = 15  # "recently crossed" window for confirmations

# === Output ===
OUT_LONG  = "long_candidates.csv"
OUT_SHORT = "short_candidates.csv"
OUT_NONE  = "no_signal_or_errors.csv"


## 3) Indicator helpers

In [3]:

def ema(series: pd.Series, length: int) -> pd.Series:
    return series.ewm(span=length, adjust=False, min_periods=length).mean()

def williams_r(h: pd.Series, l: pd.Series, c: pd.Series, length: int) -> pd.Series:
    hh = h.rolling(length).max()
    ll = l.rolling(length).min()
    return -100 * (hh - c) / (hh - ll)  # 0 to -100

def cci(h: pd.Series, l: pd.Series, c: pd.Series, length: int) -> pd.Series:
    typical = (h + l + c) / 3.0
    sma = typical.rolling(length).mean()
    mad = (typical - sma).abs().rolling(length).mean()
    return (typical - sma) / (0.015 * mad)


## 4) Strategy logic — rules & helpers

In [4]:

@dataclass
class SignalResult:
    side: str              # "LONG" or "SHORT"
    date: pd.Timestamp
    close: float
    ema50: float
    wr14: float
    cci20: float
    notes: str

def recent_cross_above(series: pd.Series, level: float, lookback: int) -> bool:
    """True if series moved from <= level to > level within 'lookback' bars and is > level now."""
    if len(series) < lookback + 2:
        return False
    wnd = series.iloc[-(lookback+1):]
    crossed_before = (wnd.shift(1) <= level).any()
    now_above = wnd.iloc[-1] > level
    return bool(crossed_before and now_above)

def recent_cross_below(series: pd.Series, level: float, lookback: int) -> bool:
    """True if series moved from >= level to < level within 'lookback' bars and is < level now."""
    if len(series) < lookback + 2:
        return False
    wnd = series.iloc[-(lookback+1):]
    crossed_before = (wnd.shift(1) >= level).any()
    now_below = wnd.iloc[-1] < level
    return bool(crossed_before and now_below)

def compute_indicators(hist: pd.DataFrame) -> pd.DataFrame:
    df = hist.copy()
    if df.empty:
        return df
    df["EMA50"] = ema(df["Close"], EMA_LEN)
    df["WR14"]  = williams_r(df["High"], df["Low"], df["Close"], WR_LEN)
    df["CCI20"] = cci(df["High"], df["Low"], df["Close"], CCI_LEN)
    return df.dropna().copy()

def scan_last_bar(df: pd.DataFrame) -> Tuple[bool, str, Optional[SignalResult]]:
    """Apply entry rules to the last bar only."""
    if df.empty: 
        return False, "empty df", None
    last = df.iloc[-1]
    close, ema50, wr, cci_val = float(last["Close"]), float(last["EMA50"]), float(last["WR14"]), float(last["CCI20"])
    wr_series = df["WR14"]
    cci_series = df["CCI20"]

    # LONG: Uptrend + momentum turn after oversold
    long_cond_trend = close > ema50
    long_cond_wr = (wr_series.iloc[-RECENT_LOOKBACK:].min() < -80) and recent_cross_above(wr_series, -50, RECENT_LOOKBACK)
    long_cond_cci = (cci_series.iloc[-RECENT_LOOKBACK:].min() < -100) and (cci_val >= -100)

    if long_cond_trend and long_cond_wr and long_cond_cci:
        return True, "LONG", SignalResult(
            "LONG", df.index[-1], close, ema50, wr, cci_val,
            "Uptrend; %R<-80 then >-50; CCI<-100 then >=-100"
        )

    # SHORT: Downtrend + momentum turn after overbought
    short_cond_trend = close < ema50
    short_cond_wr = (wr_series.iloc[-RECENT_LOOKBACK:].max() > -20) and recent_cross_below(wr_series, -50, RECENT_LOOKBACK)
    short_cond_cci = (cci_series.iloc[-RECENT_LOOKBACK:].max() > +100) and (cci_val <= +100)

    if short_cond_trend and short_cond_wr and short_cond_cci:
        return True, "SHORT", SignalResult(
            "SHORT", df.index[-1], close, ema50, wr, cci_val,
            "Downtrend; %R>-20 then <-50; CCI>+100 then <=+100"
        )

    return False, "NONE", None


## 5) Data download — yfinance helper

In [5]:

def get_hist(ticker: str, start: str) -> pd.DataFrame:
    """Download daily OHLCV for ticker since 'start'."""
    try:
        df = yf.download(ticker, start=start, progress=False, auto_adjust=False, multi_level_index=False)
        if df is None or df.empty:
            return pd.DataFrame()
        for col in ["Open","High","Low","Close","Adj Close","Volume"]:
            if col not in df.columns:
                return pd.DataFrame()
        return df
    except Exception as e:
        print(f"[ERR] {ticker}: {e}")
        return pd.DataFrame()


## 6) Run scanner — filter current signals & write CSVs

In [6]:

rows_long, rows_short, rows_none = [], [], []

for t in TICKERS:
    hist = get_hist(t, HISTORY_START)
    if hist.empty or len(hist) < MIN_BARS:
        rows_none.append({"Ticker": t, "Status": "insufficient data"})
        continue

    df = compute_indicators(hist)
    if df.empty or len(df) < MIN_BARS:
        rows_none.append({"Ticker": t, "Status": "insufficient bars post-indicator"})
        continue

    ok, side, sig = scan_last_bar(df)
    if not ok:
        rows_none.append({"Ticker": t, "Status": side})
        continue

    row = {
        "Ticker": t,
        "Date": sig.date.date(),
        "Close": round(sig.close, 4),
        "EMA50": round(sig.ema50, 4),
        "WR14": round(sig.wr14, 2),
        "CCI20": round(sig.cci20, 1),
        "Notes": sig.notes
    }
    if sig.side == "LONG":
        rows_long.append(row)
    elif sig.side == "SHORT":
        rows_short.append(row)

# Save CSVs
if rows_long:
    pd.DataFrame(rows_long).sort_values(["Date","Ticker"]).to_csv(OUT_LONG, index=False)
if rows_short:
    pd.DataFrame(rows_short).sort_values(["Date","Ticker"]).to_csv(OUT_SHORT, index=False)
if rows_none:
    pd.DataFrame(rows_none).to_csv(OUT_NONE, index=False)

# Display summaries (plain pandas)
def _show(df, title):
    print("\n" + "="*len(title))
    print(title)
    print("="*len(title))
    display(df)

if rows_long:
    _show(pd.DataFrame(rows_long).sort_values(["Date","Ticker"]), "Long candidates (current bar)")
else:
    print("No LONG candidates on the latest bar.")

if rows_short:
    _show(pd.DataFrame(rows_short).sort_values(["Date","Ticker"]), "Short candidates (current bar)")
else:
    print("No SHORT candidates on the latest bar.")

if rows_none:
    _show(pd.DataFrame(rows_none), "No-signal or errors")



Long candidates (current bar)


,Ticker,Date,Close,EMA50,WR14,CCI20,Notes
0,CANBK.NS,2025-09-15,112.56,108.9776,-8.53,114.6,Uptrend; %R<-80 then >-50; CCI<-100 then >=-100
1,ICICIGI.NS,2025-09-15,1904.00,1897.0663,-13.99,28.4,Uptrend; %R<-80 then >-50; CCI<-100 then >=-100
2,IRFC.NS,2025-09-15,128.21,127.5781,-16.44,99.3,Uptrend; %R<-80 then >-50; CCI<-100 then >=-100
4,JINDALSTEL.NS,2025-09-15,1046.95,989.5343,-8.96,90.4,Uptrend; %R<-80 then >-50; CCI<-100 then >=-100
3,JSWENERGY.NS,2025-09-15,533.10,517.7071,-3.84,86.9,Uptrend; %R<-80 then >-50; CCI<-100 then >=-100
5,VEDL.NS,2025-09-15,454.50,439.7540,-19.39,172.4,Uptrend; %R<-80 then >-50; CCI<-100 then >=-100


No SHORT candidates on the latest bar.

No-signal or errors


,Ticker,Status
0,ABB.NS,NONE
1,ADANIENSOL.NS,NONE
2,ADANIGREEN.NS,NONE
3,ADANIPOWER.NS,NONE
4,AMBUJACEM.NS,NONE
5,DMART.NS,NONE
6,BAJAJHLDNG.NS,NONE
7,BAJAJHFL.NS,insufficient bars post-indicator
8,BANKBARODA.NS,NONE
9,BPCL.NS,NONE



## 7) Notes & Adjustments

- **Lookback logic**: We check that %R/CCI reached an extreme *recently* and has now flipped back through a mid-zone threshold (−50 for %R, ±100 for CCI) in the trend’s direction.
- **Recency window**: `RECENT_LOOKBACK = 15` bars by default. Increase it to be more forgiving; decrease to be stricter.
- **Trend filter**: 50-EMA is used as the intermediate trend for a 3–4 week horizon.
- **Liquidity**: You said _no constraints_ on price/volume. You can optionally add a liquidity filter (e.g., `Volume` median > some threshold) before scanning.
- **Execution**: Signals are evaluated on the **latest bar** only to make a clean actionable list.
- **Backtest**: If you’d like, I can extend this notebook with a simple vectorized backtest with ~20 trading-day hold and protective stop.



## 8) Backtest parameters (time-based exit + ATR stops)
We'll simulate entries whenever a signal appears and exit on the earliest of:
- Max hold of `HOLD_DAYS` (≈ 3–4 weeks), or
- ATR-based stop-loss / take-profit, or
- (Optional) Trend break across EMA50 (disabled by default here for simplicity).

> The entry is at **next day's open** after a signal. P&L uses trade-level prices.


In [7]:

# === Backtest Parameters ===
HOLD_DAYS    = 20   # ~ 4 weeks
ATR_LEN      = 14
ATR_MULT_SL  = 2.0  # stop-loss multiple of ATR
ATR_MULT_TP  = 3.0  # take-profit multiple of ATR
USE_TREND_BREAK_EXIT = False  # optional extra exit (price crosses EMA50 opposite side)

def true_range(df):
    prev_close = df["Close"].shift(1)
    tr = (df["High"] - df["Low"]).abs()
    tr2 = (df["High"] - prev_close).abs()
    tr3 = (df["Low"] - prev_close).abs()
    return pd.concat([tr, tr2, tr3], axis=1).max(axis=1)

def atr(df, length=14):
    tr = true_range(df)
    return tr.rolling(length).mean()


## 9) Historical signal generator (vectorized-ish scan)

In [8]:

def generate_signals(df: pd.DataFrame) -> pd.DataFrame:
    """Mark bars as LONG/SHORT signal bars per strategy rules."""
    out = df.copy()
    out["Signal"] = "NONE"

    wr = out["WR14"]; cci = out["CCI20"]; close = out["Close"]; ema50 = out["EMA50"]
    # Windows for recency checks
    min_wr = wr.rolling(RECENT_LOOKBACK).min()
    max_wr = wr.rolling(RECENT_LOOKBACK).max()
    min_cci = cci.rolling(RECENT_LOOKBACK).min()
    max_cci = cci.rolling(RECENT_LOOKBACK).max()

    # "Cross above/below -50" approximations using 1-bar comparison
    crossed_above_m50 = (wr.shift(1) <= -50) & (wr > -50)
    crossed_below_m50 = (wr.shift(1) >= -50) & (wr < -50)

    # LONG conditions
    long_trend = close > ema50
    long_wr = (min_wr < -80) & crossed_above_m50
    long_cci = (min_cci < -100) & (cci >= -100)
    long_sig = long_trend & long_wr & long_cci

    # SHORT conditions
    short_trend = close < ema50
    short_wr = (max_wr > -20) & crossed_below_m50
    short_cci = (max_cci > +100) & (cci <= +100)
    short_sig = short_trend & short_wr & short_cci

    out.loc[long_sig, "Signal"] = "LONG"
    out.loc[short_sig, "Signal"] = "SHORT"
    return out


## 10) Backtest engine

In [9]:

def backtest_ticker(hist: pd.DataFrame, ticker: str) -> pd.DataFrame:
    """Run the backtest over all signals for a single ticker.
    Returns a trades DataFrame with entry/exit details and P&L.
    """
    df = compute_indicators(hist)
    df["ATR"] = atr(df, ATR_LEN)
    df = df.dropna().copy()
    if df.empty:
        return pd.DataFrame()

    sigdf = generate_signals(df)

    trades = []
    i = 0
    idx = sigdf.index

    while i < len(sigdf)-1:
        row = sigdf.iloc[i]
        sig = row["Signal"]
        if sig not in ("LONG","SHORT"):
            i += 1
            continue

        # Enter next day's open
        if i+1 >= len(sigdf):
            break
        entry_idx = idx[i+1]
        entry_row = sigdf.loc[entry_idx]
        entry_price = float(entry_row["Open"])
        entry_atr   = float(entry_row["ATR"])
        side = sig

        # Set stop/target
        if side == "LONG":
            stop  = entry_price - ATR_MULT_SL * entry_atr
            target= entry_price + ATR_MULT_TP * entry_atr
        else:
            stop  = entry_price + ATR_MULT_SL * entry_atr
            target= entry_price - ATR_MULT_TP * entry_atr

        # Walk forward up to HOLD_DAYS bars
        exit_idx = None
        exit_price = None
        reason = "TIME"
        for j in range(1, HOLD_DAYS+1):
            if i+1+j >= len(sigdf):
                break
            cur_idx = idx[i+1+j]
            bar = sigdf.loc[cur_idx]

            # Intra-bar stop/target approximation using High/Low vs levels
            hi, lo = float(bar["High"]), float(bar["Low"])
            if side == "LONG":
                hit_stop = lo <= stop
                hit_tgt  = hi >= target
            else:
                hit_stop = hi >= stop
                hit_tgt  = lo <= target

            if hit_stop and hit_tgt:
                # If both touched, assume worst-case (stop first) for conservatism
                exit_idx = cur_idx
                exit_price = stop
                reason = "STOP&TP_SAME_BAR->STOP"
                break
            elif hit_stop:
                exit_idx = cur_idx
                exit_price = stop
                reason = "STOP"
                break
            elif hit_tgt:
                exit_idx = cur_idx
                exit_price = target
                reason = "TP"
                break

            if USE_TREND_BREAK_EXIT:
                # Optional trend break check
                if side == "LONG" and float(bar["Close"]) < float(bar["EMA50"]):
                    exit_idx = cur_idx
                    exit_price = float(bar["Close"])
                    reason = "TREND_BREAK"
                    break
                if side == "SHORT" and float(bar["Close"]) > float(bar["EMA50"]):
                    exit_idx = cur_idx
                    exit_price = float(bar["Close"])
                    reason = "TREND_BREAK"
                    break

        # If no exit yet, exit at last checked bar close (or last available bar)
        if exit_idx is None:
            last_idx = idx[min(i+1+HOLD_DAYS, len(sigdf)-1)]
            last_bar = sigdf.loc[last_idx]
            exit_idx = last_idx
            exit_price = float(last_bar["Close"])
            reason = "TIME"

        # Compute P&L
        if side == "LONG":
            pnl = exit_price - entry_price
            ret = pnl / entry_price
        else:
            pnl = entry_price - exit_price
            ret = pnl / entry_price

        trades.append({
            "Ticker": ticker,
            "Side": side,
            "EntryDate": entry_idx.date(),
            "EntryPrice": round(entry_price, 4),
            "ExitDate": pd.Timestamp(exit_idx).date(),
            "ExitPrice": round(exit_price, 4),
            "Reason": reason,
            "Return": round(ret, 5),
            "ATR_at_Entry": round(entry_atr, 4),
        })

        # Advance the cursor to the exit bar (avoid overlapping trades)
        # Find numeric position of exit_idx
        i = sigdf.index.get_loc(exit_idx) + 1

    return pd.DataFrame(trades)


## 11) Run backtest over your universe

In [10]:

all_trades = []
for t in TICKERS:
    hist = get_hist(t, HISTORY_START)
    if hist.empty or len(hist) < MIN_BARS:
        continue
    td = backtest_ticker(hist, t)
    if not td.empty:
        all_trades.append(td)

if all_trades:
    TRADES = pd.concat(all_trades, ignore_index=True)
    TRADES.to_csv("swing_strategy_trades.csv", index=False)
    print(f"[OK] swing_strategy_trades.csv written ({len(TRADES)} trades)")

    # Basic performance by ticker
    def summarize(trades):
        if trades.empty:
            return pd.DataFrame()
        # Assume daily bar spacing; approximate daily returns series from per-trade returns over hold
        g = trades.groupby("Ticker")["Return"]
        summ = pd.DataFrame({
            "NumTrades": g.size(),
            "AvgRet": g.mean(),
            "MedRet": g.median(),
            "WinRate": (g.apply(lambda s: (s>0).mean())).round(3),
            "TotalRet": g.sum()
        })
        return summ.sort_values("TotalRet", ascending=False)

    SUMMARY = summarize(TRADES)
    SUMMARY.to_csv("swing_strategy_summary.csv")
    print("[OK] swing_strategy_summary.csv written")

    # Show in notebook
    print("\nTop 10 by Total Return:")
    display(SUMMARY.head(10))
else:
    print("No trades generated across the universe with current params.")


[OK] swing_strategy_trades.csv written (227 trades)
[OK] swing_strategy_summary.csv written

Top 10 by Total Return:


,NumTrades,AvgRet,MedRet,WinRate,TotalRet
Ticker,,,,,
LODHA.NS,5,0.098112,0.126550,0.800,0.49056
RECLTD.NS,6,0.065268,0.082370,0.667,0.39161
CGPOWER.NS,5,0.044772,0.068060,0.800,0.22386
BAJAJHLDNG.NS,5,0.040740,0.047500,0.800,0.20370
ZYDUSLIFE.NS,6,0.033452,0.057155,0.667,0.20071
BPCL.NS,5,0.032364,0.067080,0.600,0.16182
JSWENERGY.NS,5,0.030894,0.059630,0.800,0.15447
AMBUJACEM.NS,4,0.028883,0.064620,0.750,0.11553
BRITANNIA.NS,3,0.035897,0.065070,0.667,0.10769



### Notes
- **Stops/Targets:** ATR-based stops and targets approximate risk control; tweak `ATR_MULT_SL`/`ATR_MULT_TP` to fit your risk preference.
- **Execution Prices:** Entry uses **next day open** after signal; exits are approximated with **intra-bar** logic (stop/target touched by High/Low).
- **Overlapping Trades:** The engine skips ahead to the exit bar to avoid overlapping trades on the same ticker.
- **Extensions:** We can add position sizing (e.g., fixed ATR risk per trade), equity curve with drawdown, Sharpe/Sortino, and walk-forward parameter sweeps.
